In [1]:
import os

In [2]:
%pwd

'c:\\Users\\aarav\\OneDrive\\Desktop\\Cancer Detection\\Cancer-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aarav\\OneDrive\\Desktop\\Cancer Detection\\Cancer-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common  import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config



In [8]:
import os 
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e        

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
             zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-08 02:41:27,940: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-08 02:41:27,946: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-08 02:41:27,949: INFO: common: created directory at: artifacts]
[2023-12-08 02:41:27,951: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-08 02:41:27,953: INFO: 900699066: Downloading data from https://drive.google.com/file/d/1s30EKhGGPmNE541JFwEW9xFXY71e6dJ0/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1s30EKhGGPmNE541JFwEW9xFXY71e6dJ0
From (redirected): https://drive.google.com/uc?/export=download&id=1s30EKhGGPmNE541JFwEW9xFXY71e6dJ0&confirm=t&uuid=a3b24021-0075-4480-8e14-bee68d777a14
To: c:\Users\aarav\OneDrive\Desktop\Cancer Detection\Cancer-Detection\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [01:33<00:00, 525kB/s] 

[2023-12-08 02:43:07,295: INFO: 900699066: Downloaded data from https://drive.google.com/file/d/1s30EKhGGPmNE541JFwEW9xFXY71e6dJ0/view?usp=sharing into file artifacts/data_ingestion/data.zip]
